# PDI - Rebalancing data for June 2022

## Setting

In [57]:
import pandas as pd
import numpy as np
import requests

In [58]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [59]:
from defillama import DefiLlama
llama = DefiLlama()

In [60]:
import plotly.express as px
import plotly.graph_objects as go

In [61]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics** 
    1. The project must have a listing on Defi Llama
    1. The project’s token should have been listed on CoinGecko with pricing data at least 1 year prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Ethereum L1 or L2. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.

1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have at least $5mm of onchain liquidity on a single pair.
    1. The token must have shown consistent DeFi liquidity on Ethereum.

1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

#### DefiLlama

In [62]:
protocols = llama.get_all_protocols()

In [63]:
top_defi_tokens = {} 
for pr in protocols[:1000]:
    if 'Ethereum' in pr['chains'] and pr['address'] and pr['address']!='-' and pr['symbol'] != '-':
        symbol = ''.join(c for c in pr['symbol'] if c.isalnum()).upper()
        top_defi_tokens[symbol] = {
            'symbol': symbol, 
            'address': pr['address'].lower()
        }

In [64]:
top_defi_tokens['XSUSHI'] = {
    'symbol': 'XSUSHI',
    'address': "0x8798249c2e607446efb7ad49ec89dd1865ff4272",
}

#### CoinGecko

##### Tokens addresses

In [65]:
coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True))

In [66]:
tokens = {} 
tokens_by_addr = {}
for index, coin in coins_list.iterrows():
    if len(coin['platforms']) >= 1 and 'ethereum' in list(coin['platforms'].keys()):
        symbol = ''.join(c for c in coin['symbol'] if c.isalnum()).upper()
        address = coin['platforms']['ethereum'].lower()
        token = {
            'symbol': symbol,
            'address': address 
        }
        tokens[symbol] = token
        tokens_by_addr[address] = token

In [67]:
tokens['USDC'], tokens['WETH']

({'symbol': 'USDC', 'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
 {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'})

In [68]:
# These token's addresses cant be found in the coingecko list of eth addresses
exclude_symbols = []
for symb, token in top_defi_tokens.items():
    if token['address'] not in tokens_by_addr.keys():
        print(token)
        exclude_symbols.append(token['symbol'])

{'symbol': 'MULTI', 'address': '0x9fb9a33956351cf4fa040f65a13b835a3c8764e3'}
{'symbol': 'IB', 'address': 'fantom:0x00a35fd824c717879bf370e70ac6868b95870dfb'}
{'symbol': 'COW', 'address': 'heco:0x80861a817106665bca173db6ac2ab628a738c737'}
{'symbol': 'ABR', 'address': 'celo:0x6e512bfc33be36f2666754e996ff103ad1680cc9'}
{'symbol': 'FLAG', 'address': '0x57dd84042ec9507963016596a34edad42f7e4cce '}
{'symbol': 'LYRA', 'address': 'optimism:0x50c5725949a6f0c72e6c4a641f24049a917db0cb'}
{'symbol': 'SGT', 'address': '0x84810bcf08744d5862b8181f12d17bfd57d3b078'}
{'symbol': 'RSR', 'address': '0x8762db106b2c2a0bccb3a80d1ed41273552616e8'}
{'symbol': 'FHM', 'address': '0xfa1fbb8ef55a4855e5688c0ee13ac3f202486286'}
{'symbol': 'GRO', 'address': '0x09e64c2b61a5f1690ee6fbed9baf5d6990f8dfd0'}
{'symbol': 'SWPR', 'address': 'arbitrum:0xde903e2712288a1da82942dddf2c20529565ac30'}
{'symbol': 'SOLV', 'address': '0x256f2d67e52fe834726d2ddcd8413654f5eb8b53'}
{'symbol': 'HND', 'address': 'hundredfinance.jpg'}
{'symbol

In [69]:
for symb in exclude_symbols:
    del top_defi_tokens[symb]

In [70]:
# KEEP has this address in defillama, but etherscan says this address belongs to tBTC now
for token in top_defi_tokens.values():
    if token['address'] == "0x8daebade922df735c38c80c7ebd708af50815faa":
        print(token)
del top_defi_tokens['KEEP']

{'symbol': 'KEEP', 'address': '0x8daebade922df735c38c80c7ebd708af50815faa'}


##### Market data

In [71]:
cg_tokens_data = pd.DataFrame(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc'))
for i in range(2, 20):
    cg_tokens_data = cg_tokens_data.append(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', page=i))

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_81203/2628452285.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_81203/2628452285.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_81203/2628452285.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_81203/2628452285.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_81203/2628452285.py:3: FutureWarning:

The frame.append method is depreca

In [72]:
tokens_market_data = cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper())

In [73]:
tokens_market_data.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
BTC,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,30014.000000,573654853917,1,6.326629e+11,2.682979e+10,31348.000,...,2.100000e+07,21000000.0,69045.00,-56.43220,2021-11-10T14:24:11.849Z,67.810000,44261.79629,2013-07-06T00:00:00.000Z,None,2022-05-16T11:37:40.366Z
ETH,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,2041.380000,247122473074,2,NaN,1.797816e+10,2154.090,...,NaN,NaN,4878.26,-58.16207,2021-11-10T14:24:19.604Z,0.432979,471277.11907,2015-10-20T00:00:00.000Z,"{'times': 89.89250949299505, 'currency': 'btc'...",2022-05-16T11:37:35.767Z
USDT,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999322,75970665724,3,NaN,5.516610e+10,1.010,...,7.575212e+10,NaN,1.32,-24.37195,2018-07-24T00:00:00.000Z,0.572521,74.77632,2015-03-02T00:00:00.000Z,None,2022-05-16T11:37:06.137Z
USDC,usd-coin,usdc,USD Coin,https://assets.coingecko.com/coins/images/6319...,1.001000,51224512585,4,NaN,1.120949e+10,1.009,...,5.101151e+10,NaN,1.17,-14.34491,2019-05-08T00:40:28.300Z,0.891848,12.62935,2021-05-19T13:14:05.611Z,None,2022-05-16T11:36:41.143Z
BNB,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,299.790000,50454157504,5,5.045416e+10,1.712295e+09,312.540,...,1.681370e+08,168137035.9,686.31,-56.45363,2021-05-10T07:24:17.097Z,0.039818,750474.85061,2017-10-19T00:00:00.000Z,None,2022-05-16T11:38:03.053Z


### Liquidity check

In [74]:
dex_subgraph_urls = {
    'univ2': "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2",
    'sushi': "https://api.thegraph.com/subgraphs/name/zippoxer/sushiswap-subgraph-fork",
} 

In [75]:
base_tokens = [tokens['USDC'], tokens['WETH']]
base_tokens

[{'symbol': 'USDC', 'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
 {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'}]

In [76]:
query = """
    query pairs ($token0_ids: [ID!]!, $token1_ids: [ID!]!) {
        pairs (first:1000, where: {token0_in: $token0_ids, token1_in: $token1_ids}){
            id
            token0 {
              id
            }
            token1 {
              id
            }
            reserveUSD
        }
    }
    """

In [77]:
base_tokens_ids = [token['address'] for token in base_tokens]
tokens_ids = [token['address'] for symb, token in top_defi_tokens.items()]

In [78]:
rows = {token['symbol']: {} for token in top_defi_tokens.values()}
for dex_name, url in dex_subgraph_urls.items():
    request = requests.post(url, json={'query': query, 'variables': {'token0_ids': base_tokens_ids, 'token1_ids': tokens_ids}})
    pairs1 = request.json()['data']['pairs']

    request = requests.post(url, json={'query': query, 'variables': {'token0_ids': tokens_ids, 'token1_ids': base_tokens_ids}})
    pairs2 = request.json()['data']['pairs']
    
    pairs = pairs1 + pairs2
    
    print(f"{dex_name}: {len(pairs)} pairs")
    
    for pair in pairs:
        if pair['token0']['id'] in base_tokens_ids:
            base_token = tokens_by_addr[pair['token0']['id']]
            token = tokens_by_addr[pair['token1']['id']]
        else:
            base_token = tokens_by_addr[pair['token1']['id']]
            token = tokens_by_addr[pair['token0']['id']]
        
        colname = f"{dex_name}_{base_token['symbol']}" 
        
        rows[token['symbol']][colname] = float(pair['reserveUSD'])

univ2: 275 pairs
sushi: 150 pairs


In [79]:
liquidities = pd.DataFrame.from_dict(rows, orient='index')
liq_check = (liquidities > 5e6).any(axis=1).sort_values(ascending=False)
liq_check_passed = liq_check[liq_check]

In [80]:
lq_passed = tokens_market_data[
    tokens_market_data.index.isin(liq_check_passed.index)
]

In [81]:
lq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
WBTC,wrapped-bitcoin,wbtc,Wrapped Bitcoin,https://assets.coingecko.com/coins/images/7598...,30002.000000,8567911965,13,8.567912e+09,3.959176e+08,31297.000000,...,2.845040e+05,2.845040e+05,70643.000000,-57.47723,2021-11-10T14:40:19.650Z,3139.170000,856.92505,2019-04-02T00:00:00.000Z,None,2022-05-16T11:38:23.127Z
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.060000,2308777082,34,5.057677e+09,1.871744e+08,5.460000,...,1.000000e+09,1.000000e+09,44.920000,-88.75467,2021-05-03T05:25:04.822Z,1.030000,390.32483,2020-09-17T01:20:38.214Z,None,2022-05-16T11:37:43.821Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,1543.880000,1395807202,50,1.557278e+09,1.325319e+08,1636.450000,...,9.776310e+05,1.005577e+06,6292.310000,-75.44709,2021-05-03T21:54:29.333Z,168.360000,817.65772,2020-03-16T20:52:36.527Z,None,2022-05-16T11:38:13.159Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,84.540000,1164007248,60,1.357837e+09,1.957487e+08,92.090000,...,1.600000e+07,1.600000e+07,661.690000,-87.20057,2021-05-18T21:19:59.514Z,26.020000,225.45079,2020-11-05T09:20:11.928Z,None,2022-05-16T11:36:49.493Z
FXS,frax-share,fxs,Frax Share,https://assets.coingecko.com/coins/images/1342...,8.470000,512432102,110,8.428360e+08,3.093332e+07,9.330000,...,9.910795e+07,9.910795e+07,42.800000,-80.00748,2022-01-12T15:22:27.465Z,1.500000,469.25099,2021-06-25T16:50:51.447Z,None,2022-05-16T11:37:49.764Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.410000,504359959,111,1.416642e+09,5.473427e+06,1.490000,...,1.000000e+09,1.000000e+09,7.300000,-80.61149,2021-08-20T08:35:20.158Z,0.505506,180.09921,2021-03-27T09:10:29.155Z,None,2022-05-16T11:37:44.882Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,68.760000,473262690,113,6.902812e+08,4.628386e+07,74.860000,...,1.000000e+07,1.000000e+07,910.540000,-92.43816,2021-05-12T02:29:08.794Z,54.910000,25.38704,2022-05-12T07:18:59.691Z,None,2022-05-16T11:37:09.291Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,9964.520000,355344283,131,3.662813e+08,6.343329e+07,10627.080000,...,3.666600e+04,3.666600e+04,90787.000000,-89.01330,2021-05-12T00:29:37.713Z,31.650000,31419.80108,2020-07-18T12:26:27.150Z,None,2022-05-16T11:38:17.751Z
SYN,synapse-2,syn,Synapse,https://assets.coingecko.com/coins/images/1802...,1.500000,276440141,161,3.748348e+08,1.039982e+07,1.730000,...,NaN,2.500000e+08,4.920000,-69.29341,2021-10-24T00:24:41.805Z,0.389600,287.41193,2021-09-21T21:19:58.123Z,None,2022-05-16T11:38:01.416Z


### Non quantative checks

In [82]:
exclude_list = [
    {
        'symbol': 'WBTC',
        'reason': 'Criteria 1.C'
    },
    {
        'symbol': 'XSUSHI',
        'reason': 'Duplicates SUSHI'
    },
    {
        'symbol': 'ERN',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'NFTX',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'FXS',
        'reason': 'Criteria 1.G'
    },
    {
        'symbol': 'RGT',
        'reason': 'Deprecated asset'
    },
    {
        'symbol': 'SUPER',
        'reason': 'Not DeFi'
    }
]
exclude_symbols = [token['symbol'] for token in exclude_list]

In [83]:
nq_passed = lq_passed[
    ~lq_passed.index.isin(exclude_symbols)
]

In [84]:
nq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.060000,2308777082,34,5.057677e+09,1.871744e+08,5.460000,...,1.000000e+09,1.000000e+09,44.920000,-88.75467,2021-05-03T05:25:04.822Z,1.030000,390.32483,2020-09-17T01:20:38.214Z,None,2022-05-16T11:37:43.821Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,1543.880000,1395807202,50,1.557278e+09,1.325319e+08,1636.450000,...,9.776310e+05,1.005577e+06,6292.310000,-75.44709,2021-05-03T21:54:29.333Z,168.360000,817.65772,2020-03-16T20:52:36.527Z,None,2022-05-16T11:38:13.159Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,84.540000,1164007248,60,1.357837e+09,1.957487e+08,92.090000,...,1.600000e+07,1.600000e+07,661.690000,-87.20057,2021-05-18T21:19:59.514Z,26.020000,225.45079,2020-11-05T09:20:11.928Z,None,2022-05-16T11:36:49.493Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.410000,504359959,111,1.416642e+09,5.473427e+06,1.490000,...,1.000000e+09,1.000000e+09,7.300000,-80.61149,2021-08-20T08:35:20.158Z,0.505506,180.09921,2021-03-27T09:10:29.155Z,None,2022-05-16T11:37:44.882Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,68.760000,473262690,113,6.902812e+08,4.628386e+07,74.860000,...,1.000000e+07,1.000000e+07,910.540000,-92.43816,2021-05-12T02:29:08.794Z,54.910000,25.38704,2022-05-12T07:18:59.691Z,None,2022-05-16T11:37:09.291Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,9964.520000,355344283,131,3.662813e+08,6.343329e+07,10627.080000,...,3.666600e+04,3.666600e+04,90787.000000,-89.01330,2021-05-12T00:29:37.713Z,31.650000,31419.80108,2020-07-18T12:26:27.150Z,None,2022-05-16T11:38:17.751Z
SYN,synapse-2,syn,Synapse,https://assets.coingecko.com/coins/images/1802...,1.500000,276440141,161,3.748348e+08,1.039982e+07,1.730000,...,NaN,2.500000e+08,4.920000,-69.29341,2021-10-24T00:24:41.805Z,0.389600,287.41193,2021-09-21T21:19:58.123Z,None,2022-05-16T11:38:01.416Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.280000,247629929,168,3.211148e+08,1.068352e+08,1.400000,...,2.421554e+08,2.500000e+08,23.380000,-94.51440,2021-03-13T23:44:36.774Z,0.475381,169.74974,2020-11-04T14:53:53.560Z,None,2022-05-16T11:37:57.480Z
TOKE,tokemak,toke,Tokemak,https://assets.coingecko.com/coins/images/1749...,5.950000,86125152,316,5.956126e+08,3.359420e+05,6.290000,...,1.000000e+08,1.000000e+08,79.020000,-92.46856,2021-11-04T00:35:29.799Z,5.700000,4.42756,2022-05-12T18:50:54.340Z,None,2022-05-16T11:36:40.477Z


### Retrieve historical data & livetime check

In [85]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for index, data in nq_passed[['id', 'symbol']].iterrows():
    id_ = data['id']
    symbol = data['symbol'].upper()
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', symbol])
    df_prices = df_prices[df_prices[symbol] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < 365:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < 365 days')
        exclude_list.append(symbol)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', symbol])
    df_mcaps = df_mcaps[df_mcaps[symbol] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < 365:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < 365 days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Excluding SYN, prices data available only for 260 < 365 days
Excluding TOKE, prices data available only for 283 < 365 days
Excluding FLX, prices data available only for 155 < 365 days
Excluding MIST, prices data available only for 159 < 365 days


In [86]:
lt_passed = nq_passed[
    ~nq_passed.index.isin(exclude_list)
]

In [87]:
lt_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.06,2308777082,34,5.057677e+09,187174402.0,5.46,...,1.000000e+09,1.000000e+09,44.92,-88.75467,2021-05-03T05:25:04.822Z,1.030000,390.32483,2020-09-17T01:20:38.214Z,None,2022-05-16T11:37:43.821Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,1543.88,1395807202,50,1.557278e+09,132531945.0,1636.45,...,9.776310e+05,1.005577e+06,6292.31,-75.44709,2021-05-03T21:54:29.333Z,168.360000,817.65772,2020-03-16T20:52:36.527Z,None,2022-05-16T11:38:13.159Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,84.54,1164007248,60,1.357837e+09,195748694.0,92.09,...,1.600000e+07,1.600000e+07,661.69,-87.20057,2021-05-18T21:19:59.514Z,26.020000,225.45079,2020-11-05T09:20:11.928Z,None,2022-05-16T11:36:49.493Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.41,504359959,111,1.416642e+09,5473427.0,1.49,...,1.000000e+09,1.000000e+09,7.30,-80.61149,2021-08-20T08:35:20.158Z,0.505506,180.09921,2021-03-27T09:10:29.155Z,None,2022-05-16T11:37:44.882Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,68.76,473262690,113,6.902812e+08,46283862.0,74.86,...,1.000000e+07,1.000000e+07,910.54,-92.43816,2021-05-12T02:29:08.794Z,54.910000,25.38704,2022-05-12T07:18:59.691Z,None,2022-05-16T11:37:09.291Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,9964.52,355344283,131,3.662813e+08,63433294.0,10627.08,...,3.666600e+04,3.666600e+04,90787.00,-89.01330,2021-05-12T00:29:37.713Z,31.650000,31419.80108,2020-07-18T12:26:27.150Z,None,2022-05-16T11:38:17.751Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.28,247629929,168,3.211148e+08,106835241.0,1.40,...,2.421554e+08,2.500000e+08,23.38,-94.51440,2021-03-13T23:44:36.774Z,0.475381,169.74974,2020-11-04T14:53:53.560Z,None,2022-05-16T11:37:57.480Z
ALCX,alchemix,alcx,Alchemix,https://assets.coingecko.com/coins/images/1411...,33.87,44115402,473,8.121335e+07,2518283.0,36.27,...,1.994168e+06,2.393060e+06,2066.20,-98.35067,2021-03-20T19:33:19.740Z,28.550000,19.35964,2022-05-12T22:34:00.644Z,None,2022-05-16T11:36:35.773Z
BOND,barnbridge,bond,BarnBridge,https://assets.coingecko.com/coins/images/1281...,3.40,22991028,664,3.425379e+07,17781395.0,3.63,...,1.000000e+07,1.000000e+07,185.69,-98.15632,2020-10-27T00:36:32.813Z,3.060000,11.76440,2022-05-14T16:14:28.237Z,None,2022-05-16T11:37:34.033Z


### Marketcap ranking & top10 filtering

In [88]:
chosen_tokens = lt_passed.sort_values("market_cap", ascending=False).iloc[:10]

In [89]:
chosen_tokens

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.06,2308777082,34,5.057677e+09,187174402.0,5.46,...,1.000000e+09,1.000000e+09,44.92,-88.75467,2021-05-03T05:25:04.822Z,1.030000,390.32483,2020-09-17T01:20:38.214Z,None,2022-05-16T11:37:43.821Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,1543.88,1395807202,50,1.557278e+09,132531945.0,1636.45,...,9.776310e+05,1.005577e+06,6292.31,-75.44709,2021-05-03T21:54:29.333Z,168.360000,817.65772,2020-03-16T20:52:36.527Z,None,2022-05-16T11:38:13.159Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,84.54,1164007248,60,1.357837e+09,195748694.0,92.09,...,1.600000e+07,1.600000e+07,661.69,-87.20057,2021-05-18T21:19:59.514Z,26.020000,225.45079,2020-11-05T09:20:11.928Z,None,2022-05-16T11:36:49.493Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.41,504359959,111,1.416642e+09,5473427.0,1.49,...,1.000000e+09,1.000000e+09,7.30,-80.61149,2021-08-20T08:35:20.158Z,0.505506,180.09921,2021-03-27T09:10:29.155Z,None,2022-05-16T11:37:44.882Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,68.76,473262690,113,6.902812e+08,46283862.0,74.86,...,1.000000e+07,1.000000e+07,910.54,-92.43816,2021-05-12T02:29:08.794Z,54.910000,25.38704,2022-05-12T07:18:59.691Z,None,2022-05-16T11:37:09.291Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,9964.52,355344283,131,3.662813e+08,63433294.0,10627.08,...,3.666600e+04,3.666600e+04,90787.00,-89.01330,2021-05-12T00:29:37.713Z,31.650000,31419.80108,2020-07-18T12:26:27.150Z,None,2022-05-16T11:38:17.751Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.28,247629929,168,3.211148e+08,106835241.0,1.40,...,2.421554e+08,2.500000e+08,23.38,-94.51440,2021-03-13T23:44:36.774Z,0.475381,169.74974,2020-11-04T14:53:53.560Z,None,2022-05-16T11:37:57.480Z
ALCX,alchemix,alcx,Alchemix,https://assets.coingecko.com/coins/images/1411...,33.87,44115402,473,8.121335e+07,2518283.0,36.27,...,1.994168e+06,2.393060e+06,2066.20,-98.35067,2021-03-20T19:33:19.740Z,28.550000,19.35964,2022-05-12T22:34:00.644Z,None,2022-05-16T11:36:35.773Z
BOND,barnbridge,bond,BarnBridge,https://assets.coingecko.com/coins/images/1281...,3.40,22991028,664,3.425379e+07,17781395.0,3.63,...,1.000000e+07,1.000000e+07,185.69,-98.15632,2020-10-27T00:36:32.813Z,3.060000,11.76440,2022-05-14T16:14:28.237Z,None,2022-05-16T11:37:34.033Z


In [90]:
chosen_tokens_symbols = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_symbols]
marketcaps = marketcaps[chosen_tokens_symbols]

In [91]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

In [92]:
prices_data

,UNI,MKR,AAVE,LDO,COMP,YFI,SUSHI,ALCX,BOND
date,,,,,,,,,
2021-03-02,25.592699,2178.689085,386.507922,0.849448,494.358379,34900.371111,18.798040,1050.297397,56.050115
2021-03-03,25.673868,2174.258188,392.581993,0.879645,501.438114,33400.696195,17.925825,991.795750,52.260027
2021-03-04,25.486634,2206.022345,397.858343,0.836782,494.206140,33881.025287,17.517664,1376.135328,57.280345
2021-03-05,28.368215,2204.725033,376.610205,0.687990,472.457882,32560.148658,17.408828,1139.907512,55.329831
2021-03-06,28.308459,2072.617638,369.647205,0.783612,479.153858,32137.453888,16.690631,1167.196998,58.316153
...,...,...,...,...,...,...,...,...,...
2022-05-12,4.865179,1117.706814,82.394162,1.530075,68.381512,11431.432230,1.342372,32.281121,3.589246
2022-05-13,4.682426,1332.726518,77.707341,1.417807,61.330376,9945.000714,1.156318,29.419765,3.173845
2022-05-14,5.081028,1380.053908,82.426547,1.475366,68.205798,9933.794381,1.418404,31.411051,3.360034


In [93]:
marketcaps

,UNI,MKR,AAVE,LDO,COMP,YFI,SUSHI,ALCX,BOND
date,,,,,,,,,
2021-03-02,7.871645e+09,1.959551e+09,4.751222e+09,1.181185e+07,2.142571e+09,1.237769e+09,2.362758e+09,3.338203e+07,7.552599e+07
2021-03-03,7.938771e+09,1.953753e+09,4.878411e+09,1.224257e+07,2.175709e+09,1.188119e+09,2.273568e+09,3.126962e+07,7.110369e+07
2021-03-04,7.933740e+09,1.995893e+09,4.948727e+09,1.185368e+07,2.154215e+09,1.210977e+09,2.240011e+09,4.418676e+07,7.802084e+07
2021-03-05,8.803390e+09,1.982026e+09,4.684718e+09,9.733795e+06,2.054969e+09,1.156134e+09,2.211941e+09,3.647704e+07,7.517484e+07
2021-03-06,8.703822e+09,1.868843e+09,4.580840e+09,1.101787e+07,2.077348e+09,1.153659e+09,2.123790e+09,3.716741e+07,7.902313e+07
...,...,...,...,...,...,...,...,...,...
2022-05-12,2.231131e+09,1.033935e+09,1.134192e+09,5.325650e+08,4.709466e+08,4.220475e+08,2.518184e+08,4.242479e+07,2.409268e+07
2022-05-13,2.160255e+09,1.185369e+09,1.076697e+09,4.945286e+08,4.203152e+08,3.557998e+08,2.327358e+08,3.787691e+07,2.134157e+07
2022-05-14,2.310355e+09,1.238201e+09,1.137073e+09,5.181269e+08,4.667323e+08,3.513412e+08,2.419873e+08,4.056655e+07,2.257929e+07


## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 30%.
* All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
* Any asset with a weight below 0.5% will be removed from the index.


### Iterative proportional adjusting

In [94]:
def adjust_weights(w, b):
    assert abs(w.sum() - 1) < 0.01, w.sum()
    assert b * w.shape[0] >=1
    wa = w.copy()
    
    while (wa > b).any():
        c = (wa[wa>b]-b).sum()
        wa[wa > b] = b
        w_less = wa[wa < b]
        wa[wa < b] += c * (w_less / w_less.sum())
    
    return wa

### Weight mcaps

In [95]:
weights = marketcaps.div(marketcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [96]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

## Weights

### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the *check_min_weight()* function checks for any assets with a weight below the minimum threshold of 0.5% and redistributes it to the asset with the lowest valid weight (>0.5%).

In [97]:
min_weight = 0.005
adjusted_weights = weights.apply(adjust_weights, args=[0.3], axis=1)

def check_min_weight(df):
    below_min = df[df <min_weight].dropna()
    below_min_sum = below_min.sum()
    above_min = df[df>=min_weight].dropna()
    valid_min_asset = df[df>=min_weight].idxmin()
    above_min[valid_min_asset] += below_min_sum

    return above_min

adjusted_weights =adjusted_weights.apply(check_min_weight,axis=1)
adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)
adjusted_weights


,UNI,MKR,AAVE,LDO,COMP,YFI,SUSHI,ALCX,BOND
date,,,,,,,,,
2021-03-02,0.3,0.109084,0.264490,NaN,0.119272,0.075624,0.131530,NaN,NaN
2021-03-03,0.3,0.108678,0.271364,NaN,0.121025,0.072465,0.126468,NaN,NaN
2021-03-04,0.3,0.110150,0.273111,NaN,0.118887,0.074230,0.123622,NaN,NaN
2021-03-05,0.3,0.113619,0.268549,NaN,0.117800,0.073233,0.126799,NaN,NaN
2021-03-06,0.3,0.109640,0.268746,NaN,0.121872,0.075145,0.124597,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2022-05-12,0.3,0.185008,0.202947,0.095295,0.084269,0.075519,0.045059,0.011902,NaN
2022-05-13,0.3,0.216949,0.197060,0.090510,0.076927,0.065119,0.042596,0.010838,NaN
2022-05-14,0.3,0.215789,0.198165,0.090297,0.081340,0.061230,0.042173,0.011005,NaN


In [98]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

In [99]:
adjusted_weights

,UNI,MKR,AAVE,LDO,COMP,YFI,SUSHI,ALCX,BOND
date,,,,,,,,,
2021-03-02,0.3,0.109084,0.264490,NaN,0.119272,0.075624,0.131530,NaN,NaN
2021-03-03,0.3,0.108678,0.271364,NaN,0.121025,0.072465,0.126468,NaN,NaN
2021-03-04,0.3,0.110150,0.273111,NaN,0.118887,0.074230,0.123622,NaN,NaN
2021-03-05,0.3,0.113619,0.268549,NaN,0.117800,0.073233,0.126799,NaN,NaN
2021-03-06,0.3,0.109640,0.268746,NaN,0.121872,0.075145,0.124597,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2022-05-12,0.3,0.185008,0.202947,0.095295,0.084269,0.075519,0.045059,0.011902,NaN
2022-05-13,0.3,0.216949,0.197060,0.090510,0.076927,0.065119,0.042596,0.010838,NaN
2022-05-14,0.3,0.215789,0.198165,0.090297,0.081340,0.061230,0.042173,0.011005,NaN


### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [100]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)
    
    remainders = w_scaled - w_res
    
    k = round(remainders.sum())
    k_largest_rems = np.argsort(remainders)[::-1][:k]
    
    w_res[k_largest_rems] += 1
    
    return w_res 

In [101]:
last_weights = adjusted_weights.iloc[-1].dropna()
last_weights

UNI      0.300000
MKR      0.223296
AAVE     0.199891
LDO      0.083073
COMP     0.080945
YFI      0.059199
SUSHI    0.042331
ALCX     0.011266
Name: 2022-05-16 00:00:00, dtype: float64

In [102]:
converted_last_weights = convert_weights(last_weights)
converted_last_weights

UNI      76
MKR      57
AAVE     51
LDO      21
COMP     21
YFI      15
SUSHI    11
ALCX      3
Name: 2022-05-16 00:00:00, dtype: int64

## Summary table

In [103]:
pdi = pd.DataFrame()
pdi.index = chosen_tokens.index
pdi['name'] = chosen_tokens['name']
pdi['market_cap'] = marketcaps.iloc[-1].astype(int)
pdi['price'] = prices_data.iloc[-1]
pdi['weight'] = last_weights
pdi['weight_converted'] = converted_last_weights
pdi['address'] = [top_defi_tokens[symb]['address'] for symb in pdi.index]
pdi.dropna()

,name,market_cap,price,weight,weight_converted,address
symbol,,,,,,
UNI,Uniswap,2483831971,5.448515,0.300000,76.0,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
MKR,Maker,1410569552,1562.476036,0.223296,57.0,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
AAVE,Aave,1262719009,92.087935,0.199891,51.0,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
LDO,Lido DAO,524778973,1.474096,0.083073,21.0,0x5a98fcbea516cf06857215779fd812ca3bef1b32
COMP,Compound,511331521,74.861539,0.080945,21.0,0xc00e94cb662c3520282e6f5717214004a7f26888
YFI,yearn.finance,373960097,10589.620697,0.059199,15.0,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e
SUSHI,Sushi,267406373,1.391730,0.042331,11.0,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2
ALCX,Alchemix,46832340,36.141104,0.011266,3.0,0xdbdb4d16eda451d0503b854cf79d55697f90c8df
